In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import math
import numpy as np
from sklearn.metrics import classification_report
import torch.nn.functional as F

In [3]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [4]:
from transformers import (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)

I0107 07:19:02.449230 140199490098944 file_utils.py:32] TensorFlow version 2.0.0-rc1 available.
I0107 07:19:02.450446 140199490098944 file_utils.py:39] PyTorch version 1.1.0 available.


In [5]:
# Check library version
!pip list | grep -E 'transformers|torch|Keras'

Keras                2.3.1                 
Keras-Applications   1.0.8                 
Keras-Preprocessing  1.0.9                 
torch                1.1.0                 
transformers         2.2.0                 


This notebook work with env:

- Keras                2.3.1                 
- torch                1.1.0                 
- transformers         2.2.0      

# Introduction

In this notebook, will introduce how to do text classification with XLNet, including:

- Load and preprocess data
- Parser data
- Make training data
- Train model
- Evaluate result

Tips:
 - Update PyTorch to 1.1.0

**Also this notebook come with a post [Text Classification with XLNet in Action](https://medium.com/@yingbiao/text-classification-with-xlnet-in-action-869029246f7e)**<br>
**Feel free to check it, hope that it could help you.**

## Load data

**Load CSV data**

In [6]:
data_path = "data/" 

In [7]:
data_file_address = "data/text_classification_dataset.csv"

In [8]:
df_data = pd.read_csv(data_file_address,sep=",",encoding="utf-8",names=['labels','texts'])

In [9]:
df_data.columns

Index(['labels', 'texts'], dtype='object')

In [10]:
df_data.head(n=20)

,labels,texts
0,0,"god is great , the movie's not ."
1,0,. . . the whole thing succeeded only in making...
2,1,"light the candles , bring out the cake and don..."
3,1,"the story may not be new , but australian dire..."
4,1,you live the mood rather than savour the story .
5,1,". . . "" bowling for columbine "" remains a disq..."
6,1,occasionally amateurishly made but a winsome c...
7,0,"by the time you reach the finale , you're like..."
8,0,the best way to hope for any chance of enjoyin...
9,0,something must have been lost in the translati...


**Have a look labels**

In [11]:
df_data.labels.unique()

array([0, 1])

In [12]:
# Analyse the labels distribution
df_data.labels.value_counts()

1    5331
0    5331
Name: labels, dtype: int64

## Parser data

**Parser data into document structure**

In [13]:
# Get sentence data
sentences = df_data.texts.to_list()
sentences[0]

"god is great , the movie's not ."

In [14]:
# Get tag labels data
labels = df_data.labels.to_list()
print(labels[0])

0


**Make TAG name into index for training**

In [15]:
# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing
# 0:negative, 1: positive
tag2idx={'0': 0,
 '1': 1}

In [16]:
tag2idx

{'0': 0, '1': 1}

In [17]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [18]:
tag2name

{0: '0', 1: '1'}

## Make tranning data

Make raw data into trainable data for XLNet, including:

- Set gpu environment
- Load tokenizer and tokenize
- Set 3 embedding, token embedding, mask word embedding, segmentation embedding
- Split data set into train and validate, then send them to dataloader

**Set up gpu environment**

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [20]:
n_gpu

1

### Load tokenizer

Remember to install sentencepiece with  'pip install sentencepiece'

In [21]:
# Manual define vocabulary address, if you download the model in local
# The vocabulary can download from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model"
vocabulary = 'models/xlnet-base-cased/xlnet-base-cased-spiece.model'

In [22]:
# Len of the sentence must be the same as the training model
# See model's 'max_position_embeddings' = 512
max_len  = 64

In [23]:
# With cased model, set do_lower_case = False
tokenizer = XLNetTokenizer(vocab_file=vocabulary,do_lower_case=False)

### Set text input embedding

- token id embedding
- mask embedding
- segment embedding

The Embedding process was referred to [XLNet official repo](https://github.com/zihangdai/xlnet/blob/master/classifier_utils.py)


**This process is huge differnent from BERT**

In [24]:
max_len  = 64

full_input_ids = []
full_input_masks = []
full_segment_ids = []

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

UNK_ID = tokenizer.encode("<unk>")[0]
CLS_ID = tokenizer.encode("<cls>")[0]
SEP_ID = tokenizer.encode("<sep>")[0]
MASK_ID = tokenizer.encode("<mask>")[0]
EOD_ID = tokenizer.encode("<eod>")[0]

for i,sentence in enumerate(sentences):
    # Tokenize sentence to token id list
    tokens_a = tokenizer.encode(sentence)
    
    # Trim the len of text
    if(len(tokens_a)>max_len-2):
        tokens_a = tokens_a[:max_len-2]
        
        
    tokens = []
    segment_ids = []
    
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(SEG_ID_A)
        
    # Add <sep> token 
    tokens.append(SEP_ID)
    segment_ids.append(SEG_ID_A)
    
    
    # Add <cls> token
    tokens.append(CLS_ID)
    segment_ids.append(SEG_ID_CLS)
    
    input_ids = tokens
    
    # The mask has 0 for real tokens and 1 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [0] * len(input_ids)

    # Zero-pad up to the sequence length at fornt
    if len(input_ids) < max_len:
        delta_len = max_len - len(input_ids)
        input_ids = [0] * delta_len + input_ids
        input_mask = [1] * delta_len + input_mask
        segment_ids = [SEG_ID_PAD] * delta_len + segment_ids

    assert len(input_ids) == max_len
    assert len(input_mask) == max_len
    assert len(segment_ids) == max_len
    
    full_input_ids.append(input_ids)
    full_input_masks.append(input_mask)
    full_segment_ids.append(segment_ids)
    
    if 3 > i:
        print("No.:%d"%(i))
        print("sentence: %s"%(sentence))
        print("input_ids:%s"%(input_ids))
        print("attention_masks:%s"%(input_mask))
        print("segment_ids:%s"%(segment_ids))
        print("\n")
    
    
    
    

No.:0
sentence: god is great , the movie's not .
input_ids:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7290, 27, 312, 17, 19, 18, 1432, 26, 23, 50, 17, 9, 4, 3, 4, 3]
attention_masks:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
segment_ids:[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]


No.:1
sentence: . . . the whole thing succeeded only in making me groggy .
input_ids:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 9, 17, 9, 17, 9, 18, 856, 554, 5741, 114, 25, 441, 110, 17, 7059, 13006, 17, 9, 4, 3, 4, 3]
attention_m

### Set label embedding

In [25]:
# Make label into id
tags = [tag2idx[str(lab)] for lab in labels]
print(tags[0])

0


## Split data into train and validate

70% for training, 30% for validation

**Split all data**

In [26]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(full_input_ids, tags,full_input_masks,full_segment_ids, 
                                                            random_state=4, test_size=0.3)

In [27]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(7463, 3199, 7463, 3199)

**Set data into tensor**

Not recommend tensor.to(device) at this process, since it will run out of GPU memory

In [28]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

**Put data into data loader**

In [29]:
# Set batch num
batch_num = 32

In [30]:
# Set token embedding, attention embedding, segment embedding
train_data = TensorDataset(tr_inputs, tr_masks,tr_segs, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks,val_segs, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

## Train model

**Load XLNet model**

In [31]:
# In this document, contain confg(txt) and weight(bin) files
model_file_address = 'models/xlnet-base-cased'

In [32]:
# Will load config and weight with from_pretrained()
# Recommand download the model before using
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin"
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json" 
model = XLNetForSequenceClassification.from_pretrained(model_file_address,num_labels=len(tag2idx))

I0107 07:19:14.816964 140199490098944 configuration_utils.py:149] loading configuration file models/xlnet-base-cased/config.json
I0107 07:19:14.819163 140199490098944 configuration_utils.py:169] Model config {
  "attn_type": "bi",
  "bi_data": false,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "ff_activation": "gelu",
  "finetuning_task": null,
  "initializer_range": 0.02,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "n_head": 12,
  "n_layer": 12,
  "n_token": 32000,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "torchscript": false,
  "untie_r": true,
  "use_bfloat16": false
}

I0107 07:19:14.821584 140199490098944 modeling_utils.py:3

In [33]:
model;

In [34]:
# Set model to GPU,if you are using GPU machine
model.to(device);

In [35]:
# Add multi GPU support
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [36]:
# Set epoch and grad max num
epochs = 5
max_grad_norm = 1.0

In [37]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

### Set fine tuning method

**Manual optimizer**

In [38]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
# Since XLNet in 'pytorch_transformer' did not contian classifier layers
# FULL_FINETUNING = True need to set True
FULL_FINETUNING = True

In [39]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

### Fine-tuing model

In [40]:
# TRAIN loop
model.train();

In [41]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segs,b_labels = batch
        
        # forward pass
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        loss, logits = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
        

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 7463
  Batch size = 32
  Num steps = 1170


Epoch:  20%|██        | 1/5 [00:46<03:05, 46.50s/it]

Train loss: 0.4623915070244171


Epoch:  40%|████      | 2/5 [01:33<02:19, 46.51s/it]

...

Train loss: 0.18195790735117356


Epoch:  80%|████████  | 4/5 [03:05<00:46, 46.48s/it]

Train loss: 0.12496821583116771


Epoch: 100%|██████████| 5/5 [03:52<00:00, 46.48s/it]

Train loss: 0.08649337041899242


## Save model 

In [42]:
xlnet_out_address = 'models/xlnet_out_model/tc02'

In [43]:
# Make dir if not exits
if not os.path.exists(xlnet_out_address):
        os.makedirs(xlnet_out_address)

In [44]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [45]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(xlnet_out_address, "pytorch_model.bin")
output_config_file = os.path.join(xlnet_out_address, "config.json")

In [46]:
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(xlnet_out_address)

('models/xlnet_out_model/tc02/spiece.model',)

## Load model

In [47]:
model = XLNetForSequenceClassification.from_pretrained(xlnet_out_address,num_labels=len(tag2idx))

I0107 07:23:15.511597 140199490098944 configuration_utils.py:149] loading configuration file models/xlnet_out_model/tc02/config.json
I0107 07:23:15.513851 140199490098944 configuration_utils.py:169] Model config {
  "attn_type": "bi",
  "bi_data": false,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "ff_activation": "gelu",
  "finetuning_task": null,
  "initializer_range": 0.02,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "n_head": 12,
  "n_layer": 12,
  "n_token": 32000,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "torchscript": false,
  "untie_r": true,
  "use_bfloat16": false
}

I0107 07:23:15.515127 140199490098944 modeling_utils.

In [48]:
# Set model to GPU
model.to(device);

In [49]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

## Eval model

In [50]:
# Evalue loop
model.eval();

In [51]:
# Set acc funtion
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [52]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

y_true = []
y_predict = []
print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segs,b_labels = batch
    
    with torch.no_grad():
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        tmp_eval_loss, logits = outputs[:2]
    
    # Get textclassification predict result
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)
#     print(tmp_eval_accuracy)
#     print(np.argmax(logits, axis=1))
#     print(label_ids)
    
    # Save predict and real label reuslt for analyze
    for predict in np.argmax(logits, axis=1):
        y_predict.append(predict)
        
    for real_result in label_ids.tolist():
        y_true.append(real_result)

    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
   
    nb_eval_steps += 1
    
    
eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / len(val_inputs)
loss = tr_loss/nb_tr_steps 
result = {'eval_loss': eval_loss,
                  'eval_accuracy': eval_accuracy,
                  'loss': loss}
report = classification_report(y_pred=np.array(y_predict),y_true=np.array(y_true))

# Save the report into file
output_eval_file = os.path.join(xlnet_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
        print("  %s = %s"%(key, str(result[key])))
        writer.write("%s = %s\n" % (key, str(result[key])))
        
    print(report)
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =3199
  Batch size = 32
***** Eval results *****
  eval_accuracy = 0.8783994998437011
  eval_loss = 0.5584006253629923
  loss = 0.08649337041899242
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      1586
           1       0.86      0.90      0.88      1613

    accuracy                           0.88      3199
   macro avg       0.88      0.88      0.88      3199
weighted avg       0.88      0.88      0.88      3199

